In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import math
import pandas as pd
import numpy as np

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
driver = webdriver.Chrome(executable_path=r"C:\users\liber\chromedriver.exe", options=options)
driver.get("http://bvmf.bmfbovespa.com.br/cias-listadas/empresas-listadas/BuscaEmpresaListada.aspx?idioma=pt-br")
time.sleep(10)
driver.find_element_by_xpath('//*[@id="ctl00_contentPlaceHolderConteudo_BuscaNomeEmpresa1_btnTodas"]').click()
print('Site da B3 aberto com sucesso.')

l_colunas = ['cnpj', 'nome_cia', 'ativ_princ', 'receita_2019', 'res_antes_tl_2019', 'ir_corr_2019', 'ativo_total_2019', 
             'pl_2019', 'receita_2018', 'res_antes_tl_2018', 'ir_corr_2018', 'ativo_total_2018', 'pl_2018', 'receita_2017', 
             'res_antes_tl_2017', 'ir_corr_2017', 'ativo_total_2017', 'pl_2017', 'receita_2016', 'res_antes_tl_2016', 
             'ir_corr_2016', 'ativo_total_2016', 'pl_2016', 'receita_2015', 'res_antes_tl_2015', 'ir_corr_2015', 
             'ativo_total_2015', 'pl_2015', 'receita_2014', 'res_antes_tl_2014', 'ir_corr_2014', 'ativo_total_2014', 
             'pl_2014', 'receita_2013', 'res_antes_tl_2013', 'ir_corr_2013', 'ativo_total_2013', 'pl_2013', 'receita_2012', 
             'res_antes_tl_2012', 'ir_corr_2012', 'ativo_total_2012', 'pl_2012', 'receita_2011', 'res_antes_tl_2011', 
             'ir_corr_2011', 'ativo_total_2011', 'pl_2011']

l_registros = []

WebDriverWait(driver, 30).until(EC.element_to_be_clickable((
    By.XPATH, '/html/body/form/div[3]/div[1]/div/div/div/div/div[2]/div[1]/div[2]/div/table/tbody/tr')))
linhas = driver.find_elements_by_xpath('/html/body/form/div[3]/div[1]/div/div/div/div/div[2]/div[1]/div[2]/div/table/tbody/tr')
qtd_linhas = len(linhas)

for i in range(32, qtd_linhas):
    #Incicializa o dicionário de dados a cada mudança de companhia
    print('Recuperando os dados da ' + str(i+1) + 'ª empresa...')
    d_dados = {}
    d_dados['cnpj'], d_dados['nome_cia'], d_dados['ativ_princ'] = "", "", ""
    d_dados['receita_2019'], d_dados['res_antes_tl_2019'], d_dados['ir_corr_2019'] = 0, 0, 0
    d_dados['ativo_total_2019'], d_dados['pl_2019'] = 0, 0
    d_dados['receita_2018'], d_dados['res_antes_tl_2018'], d_dados['ir_corr_2018'] = 0, 0, 0
    d_dados['ativo_total_2018'], d_dados['pl_2018'] = 0, 0
    d_dados['receita_2017'], d_dados['res_antes_tl_2017'], d_dados['ir_corr_2017'] = 0, 0, 0
    d_dados['ativo_total_2017'], d_dados['pl_2017'] = 0, 0
    d_dados['receita_2016'], d_dados['res_antes_tl_2016'], d_dados['ir_corr_2016'] = 0, 0, 0
    d_dados['ativo_total_2016'], d_dados['pl_2016'] = 0, 0
    d_dados['receita_2015'], d_dados['res_antes_tl_2015'], d_dados['ir_corr_2015'] = 0, 0, 0
    d_dados['ativo_total_2015'], d_dados['pl_2015'] = 0, 0
    d_dados['receita_2014'], d_dados['res_antes_tl_2014'], d_dados['ir_corr_2014'] = 0, 0, 0
    d_dados['ativo_total_2014'], d_dados['pl_2014'] = 0, 0
    d_dados['receita_2013'], d_dados['res_antes_tl_2013'], d_dados['ir_corr_2013'] = 0, 0, 0
    d_dados['ativo_total_2013'], d_dados['pl_2013'] = 0, 0
    d_dados['receita_2012'], d_dados['res_antes_tl_2012'], d_dados['ir_corr_2012'] = 0, 0, 0
    d_dados['ativo_total_2012'], d_dados['pl_2012'] = 0, 0
    d_dados['receita_2011'], d_dados['res_antes_tl_2011'], d_dados['ir_corr_2011'] = 0, 0, 0
    d_dados['ativo_total_2011'], d_dados['pl_2011'] = 0, 0
    
    #Recupera os dados cadastrais
    emp = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((
        By.XPATH, '//*[@id="ctl00_contentPlaceHolderConteudo_BuscaNomeEmpresa1_grdEmpresa_ctl01"]/tbody/tr[' 
        + str(i+1) + ']/td[1]/a')))
    emp.click()
    
    elem_nome_cia = driver.find_element_by_xpath(
        '//*[@id="ctl00_contentPlaceHolderConteudo_MenuEmpresasListadas1_lblNomeEmpresa"]')
    d_dados['nome_cia'] = elem_nome_cia.text
    
    driver.switch_to.frame("ctl00_contentPlaceHolderConteudo_iframeCarregadorPaginaExterna")
    elem_cnpj = WebDriverWait(driver, 30).until(EC.presence_of_element_located((
        By.XPATH, '/html/body/div[2]/div[1]/ul/li[1]/div/table/tbody/tr[3]/td[2]')))
    d_dados['cnpj'] = elem_cnpj.text
    
    elem_ativ_princ = WebDriverWait(driver, 30).until(EC.presence_of_element_located((
        By.XPATH, '/html/body/div[2]/div[1]/ul/li[1]/div/table/tbody/tr[4]/td[2]')))
    d_dados['ativ_princ'] = elem_ativ_princ.text  
    
    driver.switch_to.default_content()
    
    #Abre a última Demonstração Financeira Patronizada (DFP) disponível (em regra, 2019), que normalmente contém os dados 
    #dos 3 últimos exercícios. Se não contiver dos 3 últimos, deve conter dos 2 últimos. A variável prox_dfp depende 
    #da existência (ou não) da terceira coluna na tabela
    area_dfp = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((
        By.XPATH, '//*[@id="ctl00_contentPlaceHolderConteudo_MenuEmpresasListadas1_tabMenuEmpresa_tabRelatoriosFinanceiros"]/span/span')))
    area_dfp.click()
    
    #Inicializa um loop para recuperar, além dos dados da última DFP (em regra, 2019), também os dados da DFP de 2016 
    # (se existente), que contém os dados de 2014 a 2016, e também da DFP de 2013, que contém os dados de 2011 a 2013, 
    # fazendo com que, ao todo, sejam recuperados os dados de 2011 a 2019.
    prox_dfp = '2019'
    j = 0
    while j < 3:
        dfp = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((
            By.XPATH, '//*[@id="ctl00_contentPlaceHolderConteudo_rptDocumentosDFP_ctl00_lnkDocumento"]')))
        dfp.click()
        if prox_dfp != '2019':
            driver.find_element_by_xpath('//*[@id="ctl00_botaoNavegacaoVoltar"]').click()

        janelas = driver.window_handles
        driver.switch_to.window(janelas[1])
        WebDriverWait(driver, 30).until(EC.frame_to_be_available_and_switch_to_it((
            By.ID, 'iFrameFormulariosFilho')))
        
        #Se a tabela existir na página aberta, recupera os dados da DRE: Receita Líquida, Resultado antes dos Tributos 
        #sobre o Lucro, e Imposto de Renda Corrente   
        tab_existe = driver.find_elements_by_xpath('/html/body/form/div[4]/table/tbody')
        if len(tab_existe) > 0:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((
                By.XPATH, '/html/body/form/div[4]/table/tbody')))
            tabela = driver.find_element_by_xpath('/html/body/form/div[4]/table/tbody')        
        else:
            driver.close()
            driver.switch_to.window(janelas[0])
            if prox_dfp != '2019':
                prox_dfp = str(int(prox_dfp)+1)
            else:
                prox_dfp = str(int(prox_dfp)-1)                
            drop_box = Select(driver.find_element_by_xpath(
                '/html/body/form/div[3]/div/div[1]/div/div[3]/div[2]/div[1]/div/select'))
            drop_box.select_by_visible_text(prox_dfp)
            continue
        for item in tabela.find_elements_by_xpath('.//tr'):
            coluna = []
            for col in item.find_elements_by_xpath('.//td'):
                coluna.append(col.text)
            if coluna[0] == " Conta ":
                ano3 = coluna[2][-5:-1]
                if len(coluna) == 3:
                    prox_dfp = str(int(prox_dfp)-1)
                elif len(coluna) == 4:
                    prox_dfp = str(int(prox_dfp)-2)
                    ano2 = coluna[3][-5:-1]
                elif len(coluna) == 5:
                    prox_dfp = str(int(prox_dfp)-3)
                    ano2 = coluna[3][-5:-1]                    
                    ano1 = coluna[4][-5:-1]
            if coluna[0] == " 3.01 ":
                d_dados['receita_'+ano3] = int(coluna[2].replace('.', '').strip(' ')) if not coluna[2].isspace() else 0
                if len(coluna) >= 4:
                    d_dados['receita_'+ano2] = int(coluna[3].replace('.', '').strip(' ')) if not coluna[3].isspace() else 0
                if len(coluna) == 5:
                    d_dados['receita_'+ano1] = int(coluna[4].replace('.', '').strip(' ')) if not coluna[4].isspace() else 0
            if coluna[0] == " 3.07 ":
                d_dados['res_antes_tl_'+ano3] = int(coluna[2].replace('.', '').strip(' ')) if not coluna[2].isspace() else 0
                if len(coluna) >= 4:
                    d_dados['res_antes_tl_'+ano2] = int(coluna[3].replace('.', '').strip(' ')) if not coluna[3].isspace() else 0
                if len(coluna) == 5:
                    d_dados['res_antes_tl_'+ano1] = int(coluna[4].replace('.', '').strip(' ')) if not coluna[4].isspace() else 0
            if coluna[0] == " 3.08.01 ":
                d_dados['ir_corr_'+ano3] = int(coluna[2].replace('.', '').strip(' '))*(-1) if not coluna[2].isspace() else 0
                if len(coluna) >= 4:
                    d_dados['ir_corr_'+ano2] = int(coluna[3].replace('.', '').strip(' '))*(-1) if not coluna[3].isspace() else 0
                if len(coluna) == 5:
                    d_dados['ir_corr_'+ano1] = int(coluna[4].replace('.', '').strip(' '))*(-1) if not coluna[4].isspace() else 0
        
        #Recupera os dados do Balanço Patrimonial, nas contas de Passivo. Como a norma contábil determina que o Ativo Total 
        #deve ser igual ao Passivo Total, por economia recuperamos o valor do passivo, mas damos o label de Ativo, já que são 
        #idênticos. Recuperamos também o valor do Patrimônio Líquido
        driver.switch_to.default_content()
        drop_box = Select(driver.find_element_by_xpath('/html/body/form/div[5]/div/select[2]'))
        drop_box.select_by_visible_text('Balanço Patrimonial Passivo')

        WebDriverWait(driver, 30).until(EC.frame_to_be_available_and_switch_to_it((
            By.ID, 'iFrameFormulariosFilho')))

        WebDriverWait(driver, 30).until(EC.presence_of_element_located((
            By.XPATH, '/html/body/form/div[4]/table/tbody')))
        tabela = driver.find_element_by_xpath('/html/body/form/div[4]/table/tbody')
        for item in tabela.find_elements_by_xpath('.//tr'):
            coluna = []
            for col in item.find_elements_by_xpath('.//td'):
                coluna.append(col.text)
            if coluna[0] == " Conta ":
                ano3 = coluna[2][-5:-1]
                if len(coluna) >= 4:
                    ano2 = coluna[3][-5:-1]
                if len(coluna) == 5:
                    ano1 = coluna[4][-5:-1]
            if coluna[0] == " 2 ":
                d_dados['ativo_total_'+ano3] = int(coluna[2].replace('.', '').strip(' ')) if not coluna[2].isspace() else 0
                if len(coluna) >= 4:
                    d_dados['ativo_total_'+ano2] = int(coluna[3].replace('.', '').strip(' ')) if not coluna[3].isspace() else 0
                if len(coluna) == 5:
                    d_dados['ativo_total_'+ano1] = int(coluna[4].replace('.', '').strip(' ')) if not coluna[4].isspace() else 0
            if coluna[0] == " 2.03 ":
                d_dados['pl_'+ano3] = int(coluna[2].replace('.', '').strip(' ')) if not coluna[2].isspace() else 0
                if len(coluna) >= 4:
                    d_dados['pl_'+ano2] = int(coluna[3].replace('.', '').strip(' ')) if not coluna[3].isspace() else 0
                if len(coluna) == 5:
                    d_dados['pl_'+ano1] = int(coluna[4].replace('.', '').strip(' ')) if not coluna[4].isspace() else 0
        
        driver.close()    
        driver.switch_to.window(janelas[0])
        if j in range(2):
            lista_opcoes = []
            opcoes = driver.find_elements_by_xpath('//*[@id="ctl00_contentPlaceHolderConteudo_cmbAno"]')
            for ano in opcoes:
                lista_opcoes = ano.text.replace(" ", "").split('\n')
            if prox_dfp in ['2019', '2018', '2017', '2016', '2015', '2014', '2013'] and prox_dfp in lista_opcoes:
                drop_box = Select(driver.find_element_by_xpath(
                    '/html/body/form/div[3]/div/div[1]/div/div[3]/div[2]/div[1]/div/select'))
                drop_box.select_by_visible_text(prox_dfp)
            else:
                break
    j = j+1
    
    #Mantém os dados recuperados em uma lista de tuplas
    l_registros.append(tuple(d_dados.values()))

    driver.find_element_by_xpath('//*[@id="ctl00_botaoNavegacaoVoltar"]').click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="ctl00_botaoNavegacaoVoltar"]').click()
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="ctl00_contentPlaceHolderConteudo_BuscaNomeEmpresa1_btnTodas"]').click()
    
    time.sleep(5)

driver.quit()

#Exporta o resultado para um arquivo CSV
df_b3 = pd.DataFrame(l_registros, columns = l_colunas)
df_b3.to_csv('dados_B3.csv', index = False)
df_b3.head()

Site da B3 aberto com sucesso.
Recuperando os dados da 33ª empresa...
Recuperando os dados da 34ª empresa...
Recuperando os dados da 35ª empresa...
Recuperando os dados da 36ª empresa...
Recuperando os dados da 37ª empresa...


KeyboardInterrupt: 

In [4]:
df_b3 = pd.DataFrame(l_registros, columns = l_colunas)
df_b3.to_csv('dados_B3.csv', index = False)
df_b3

,cnpj,nome_cia,ativ_princ,receita_2019,res_antes_tl_2019,ir_corr_2019,ativo_total_2019,pl_2019,receita_2018,res_antes_tl_2018,...,receita_2012,res_antes_tl_2012,ir_corr_2012,ativo_total_2012,pl_2012,receita_2011,res_antes_tl_2011,ir_corr_2011,ativo_total_2011,pl_2011
0,60.851.615/0001-53,BARDELLA S.A. INDUSTRIAS MECANICAS,Bens de Capital. Produtos E Serviços Para Os S...,38038,-249245,0,728221,-97690,93699,-98167,...,310080,-34906,71,713695,410179,513041,40482,3459,788355,450039
1,42.331.462/0001-31,BATTISTELLA ADM PARTICIPACOES S.A.,Com.de Caminhões E ónibus Scania. Seus Acessór...,470723,11594,1925,130513,10087,358889,-11877,...,789062,-27130,7224,289696,-65314,939865,-28782,20341,360063,-30533
2,61.374.161/0001-30,BAUMER S.A.,Fabricação Aparelhos E Instrumentos P/ Uso Med...,149310,9428,5234,190587,108127,119802,5931,...,99746,14152,5098,100514,62735,93512,10049,1879,88433,53582
3,17.344.597/0001-94,BB SEGURIDADE PARTICIPAÇÕES S.A.,Participação No Capital Social de Outras Socie...,0,8600935,-343522,14926547,5248754,0,4414393,...,0,0,0,7292611,5638374,0,0,0,0,0


In [ ]:
dados_b3 = pd.read_csv('dados_B3.csv')
dados_b3

In [ ]:
driver.quit()